# Tutorial getting started

This notebook is an introduction on how-to use [HYDROLIB-core](https://github.com/Deltares/HYDROLIB-core) functionalities in [HYDROLIB](https://github.com/Deltares/HYDROLIB) scripts or other Python scripts. 

```HYDROLIB```is a Python package with tools for pre-processing, post-processing and analysis of hydrodynamical data and simulation results, currently focused on D-HYDRO Suite 1D2D. HYDROLIB  builds upon the basic ```HYDROLIB-core``` Python package. This is the core library of Python wrappers around the D-HYDRO Suite 1D2D model files (input and output) and model engines (kernel libraries).

The goal of this tutorial is to familiarize HYDROLIB developers with how to apply HYDROLIB-core functionalities in their scripts. Hence this notebook consist of five parts: 
1. Read model and/or -elements
2. Adjust model parameters of existing model
3. Write model(elements) I
4. Extend existing model with new elements
5. Write model(elements) II

To run this notebook a simple Python environment with the following packages is required:
```
pip install hydrolib-core pandas pathlib
```
_Remark_ Note that an email was sent to all participants to create an environment with the ```hydrolib-core``` package installed. In the meantime, a new release was created, so please update the pip install of this package:
```
pip install -U hydrolib-core
```

Furthermore, a dummy model is needed to run the notebook code. This dummy-model is generated using the D-HyDAMO converter scripts ([delft3dfmpy](https://github.com/openearth/delft3dfmpy) and will, for now, be provided to you via a download link. 

Let's get started

## Loading libraries
__EX1__: Import the libraries required in the notebook below. _Tip: please consult the [HYDROLIB-core documentation](https://deltares.github.io/HYDROLIB-core/tutorials/steps/)_

In [16]:
# basis 
import pandas as pd

# I/O
from pathlib import Path

# Answer to 1 -  hydrolib-core libraries
from hydrolib.core.io.dimr.models import DIMR, FMComponent
from hydrolib.core.io.mdu.models import FMModel
from hydrolib.core.io.structure.models import FlowDirection, StructureModel, Weir



## Read model and/or -elements
This part shows the different ways of reading model and/or -elements. 

There are 3 levels at which a model and/or its elements can be read:
1. DIMR-level: 
```python
dimr = DIMR(directory_to_dimr_config.xml)
```
2. MDU-level: 
```python
fm = FMModel(directory_to_MDU-file)
```
3. file-level (element-level): e.g. structure INI level
```python
struc = StructureModel(directory_to_structure-ini)
```

Most classes in `hydrolib.core` are hierarchically organized by file type, and can be accessed via `hydrolib.core.io.<fileext/type>.models`.

__EX2__: Read the model using the MDU-level option. _Tip: (i) specify path and filenames, (ii) see above + HYDROLIB-core documentation_

In [2]:
# Answer EX2

# Specify paths
root = Path.cwd()
inputdir = root / "data"
modeldir = inputdir / "model"

# Specify filenames
mdufile = "moergestels_broek.mdu"

# Read mdu model
fm = FMModel(modeldir / mdufile)

__EX3__: Please print (a) MDU-model, (b) tree of MDU-model (_tip:see [principles of HYDROLIB-core](https://deltares.github.io/HYDROLIB-core/topics/principles/)_), (c) Print geometry element of MDU-model, and (d)
get filename of structures INI-file (_tip: use tab to see different options_)

In [15]:
# Answer EX3

# 3a simple print
print(fm)

# 3b Get tree of fm variable
fm.show_tree()

# 3c Print geometry elements of MDU-model
#print(fm.geometry)

# 3d Get filename of structure INI-file
print(fm.geometry.structurefile)
structuresfile = fm.geometry.structurefile[0].filepath
print(structuresfile)



d:\dam_ar\unstruc_plannen\2021\20210000_HYDROLIB\N\E. Correspondence\Meetings\20211102_HYDROLIB-core_developer_training\getting_started\output\moergestels_broek.mdu
  d:\dam_ar\unstruc_plannen\2021\20210000_HYDROLIB\N\E. Correspondence\Meetings\20211102_HYDROLIB-core_developer_training\getting_started\output\moergestels_broek.mdu
    Geometry
     ∟ d:\dam_ar\unstruc_plannen\2021\20210000_HYDROLIB\N\E. Correspondence\Meetings\20211102_HYDROLIB-core_developer_training\getting_started\output\moergestels_broek_net.nc
     ∟ d:\dam_ar\unstruc_plannen\2021\20210000_HYDROLIB\N\E. Correspondence\Meetings\20211102_HYDROLIB-core_developer_training\getting_started\output\structures.ini
     ∟ d:\dam_ar\unstruc_plannen\2021\20210000_HYDROLIB\N\E. Correspondence\Meetings\20211102_HYDROLIB-core_developer_training\getting_started\output\r001.ini
     ∟ d:\dam_ar\unstruc_plannen\2021\20210000_HYDROLIB\N\E. Correspondence\Meetings\20211102_HYDROLIB-core_developer_training\getting_started\output\r002.i

(i) Note that the header and parameter names from the MDU-file are the same ones use in the Python objects and their fields to get to the data.

(ii) Note that if multiple structure files are specified in the MDU-file, a for-loop (list comprehension) would be needed. Even for a single structure file, the `fm.geometry.structurefile` field is always a list.

__EX4__: Get structure data

(i) Note that the `structurefile[0].structure` field is a list, for all structures in the underlying file.

In [4]:
# Answer to exercise 4 
# 4a Get the actual list of structures in the file
struc_A = fm.geometry.structurefile[0].structure
print(struc_A)

[Weir(comments=Comments(id=None, name=None, branchid=None, chainage=None, numcoordinates='Number of values in xCoordinates and yCoordinates', xcoordinates='x-coordinates of the location of the structure. (number of values = numCoordinates)', ycoordinates='y-coordinates of the location of the structure. (number of values = numCoordinates)', type=None, allowedflowdir=None, crestlevel=None, crestwidth=None, corrcoeff=None, usevelocityheight=None, crest=None), id='Weir_RS373-st1', name='Weir_RS373-st1', type='weir', branchid='riv_RS373_2442', chainage=584.463, numcoordinates=None, xcoordinates=None, ycoordinates=None, allowedflowdir='both', crestlevel=10.429, crestwidth=0.71, corrcoeff=1.0, usevelocityheight=True, crest='boundaries.bc', datablock=None, _header='Structure'), Weir(comments=Comments(id=None, name=None, branchid=None, chainage=None, numcoordinates='Number of values in xCoordinates and yCoordinates', xcoordinates='x-coordinates of the location of the structure. (number of value

__IMPORTANT__: For some cases, it is not necessary to read the entire model. Reading the specific model file would be sufficient. 

__EX5__: (a) Read structure INI-File without reading the entire model. (b) Get structure data (save in other variable than EX4)

In [5]:
# Answer to exercise 5:

# 5a Read structure INI-file
structures = StructureModel(structuresfile)

# 5b Get structure data again in this file
struc_B = structures.structure
print(struc_B)

[Weir(comments=Comments(id=None, name=None, branchid=None, chainage=None, numcoordinates='Number of values in xCoordinates and yCoordinates', xcoordinates='x-coordinates of the location of the structure. (number of values = numCoordinates)', ycoordinates='y-coordinates of the location of the structure. (number of values = numCoordinates)', type=None, allowedflowdir=None, crestlevel=None, crestwidth=None, corrcoeff=None, usevelocityheight=None, crest=None), id='Weir_RS373-st1', name='Weir_RS373-st1', type='weir', branchid='riv_RS373_2442', chainage=584.463, numcoordinates=None, xcoordinates=None, ycoordinates=None, allowedflowdir='both', crestlevel=10.429, crestwidth=0.71, corrcoeff=1.0, usevelocityheight=True, crest='boundaries.bc', datablock=None, _header='Structure'), Weir(comments=Comments(id=None, name=None, branchid=None, chainage=None, numcoordinates='Number of values in xCoordinates and yCoordinates', xcoordinates='x-coordinates of the location of the structure. (number of value

## Adjust model parameters of existing model
This part shows:
* how to get the model data to a dataframe, 
* adjust the model data.
* and how to convert the model back to a `hydrolib.core` Pydantic object

Note that this is the step where you could connect to your HYDROLIB script.

__Ex6__: Convert the structure data of both reading option to a dataframe. _Tip: please check [dataframe handling example](https://deltares.github.io/HYDROLIB-core/tutorials/dataframe_handling/)_

In [6]:
# Answer to 6

# Dict to df
df_struc_A = pd.DataFrame([f.__dict__ for f in struc_A])
df_struc_B = pd.DataFrame([f.__dict__ for f in struc_B])

__EX7__: Please investigate the dataframes. What do you notice?

In [7]:
# Answer to 7
print(df_struc_A.head())
print(df_struc_A.columns)
print(df_struc_A.index)
print(df_struc_A["type"].unique())

# You can see the following things:
# (i) all structures are in one datafile, 
# (ii) this model has the following structures types: weir, universal weir, bridge, pump and compound, 
# (iii) 12 structures are in the model
# (iv) the columns names are all structure parameters in lowercase
# (v) and only the relevant structure columns are filled for each structure. 



                                            comments              id  \
0  id=None name=None branchid=None chainage=None ...  Weir_RS373-st1   
1  id=None name=None branchid=None chainage=None ...  Weir_RS373-st2   
2  id=None name=None branchid=None chainage=None ...   Weir_RS1-st15   
3  id=None name=None branchid=None chainage=None ...      extra_weir   
4  id=None name=None branchid=None chainage=None ...       RS1-KBR32   

             name    type        branchid  chainage numcoordinates  \
0  Weir_RS373-st1    weir  riv_RS373_2442   584.463           None   
1  Weir_RS373-st2    weir    riv_RS375_41   573.639           None   
2   Weir_RS1-st15    weir    riv_RS1_1810   977.222           None   
3      extra_weir    weir    riv_RS1_1810   950.000           None   
4       RS1-KBR32  bridge    riv_RS375_41   701.141           None   

  xcoordinates ycoordinates allowedflowdir  ...  yvalues  zvalues  \
0         None         None           both  ...      NaN      NaN   
1       

__EX8__: Change the crestlevel and crestwidth of the weir "Weir_RS373-st1" to respectively 10.1 m AD and 0.69 m in the dataframes.

In [8]:
# Answer to 8

# Adjust parameters, for example a crest level
df_struc_A.loc[df_struc_A["id"] == "Weir_RS373-st1", "crestlevel"] = 10.1
df_struc_A.loc[df_struc_A["id"] == "Weir_RS373-st1", "crestwidth"] = 0.69

df_struc_B.loc[df_struc_B["id"] == "Weir_RS373-st1", "crestlevel"] = 10.1
df_struc_B.loc[df_struc_B["id"] == "Weir_RS373-st1", "crestwidth"] = 0.69



__EX9__: Convert the dataframes to a Pydantic object. Please note that the structures in the MDU-Model should be replaced.

In [9]:
# Answer to 9

# Convert dataframe back to object
struc_A = StructureModel(structure=df_struc_A.to_dict("records"))
struc_B = StructureModel(structure=df_struc_B.to_dict("records"))

# Store struc_A in FMModel
fm.geometry.structurefile[0] = struc_A




Now we learned how to adjust the model parameters in the structure INI-file. Below you will practise with adjusting model parameters in the MDU-file self. 

__EX10__: Adjust the stop time of the simulation to 2 days after the start time. (_Tip: Use header and parameter name from MDU-file_ to access the model fields in the Python objects.)

In [10]:
# Answer to 10
print(fm.time.tstart)
print(fm.time.tstop)
fm.time.tstop = 2*fm.time.tstop
print(fm.time.tstop)

0.0
86400.0
172800.0


# Write model(elements) I

Similar to the reading part, there are also three levels at which the model and/or -elements can be saved. This can be done with the following function: 
```python
"Model(element)".save(output_directory)
```
In this part we will do it for model level (MDU-file) and file level (structure INI-file). 

__EX11__: Create output directory for (a) model, and (b) modelfiles

In [11]:
# Answer to 11

# Create output directories
outputdir = root / "output"
outputdir_files = root / "output_files"
outputdir.mkdir(parents=True, exist_ok=True)
outputdir_files.mkdir(parents=True, exist_ok=True)


__EX12__: Save/write adjusted (a) MDU-model, (b) structure INI-file, and (c) check the saved models

In [12]:
# Answer to 12

# Save MDU-Model
fm.save(outputdir)

# Save Structure INI-file
struc_B.save(outputdir_files)

WindowsPath('d:/dam_ar/unstruc_plannen/2021/20210000_HYDROLIB/N/E. Correspondence/Meetings/20211102_HYDROLIB-core_developer_training/getting_started/output_files/structures.ini')

# Extend existing model with new elements

In some cases, new elements needs to be added to an existing model and/or model file or even an entire new model should be generated. 

This part shows  how to add a single new element to an existing model. First an new weir will be added, followed by turning the MDU-model into a DIMR model and adding the dimr_config.xml into the mix. 

__EX13__:Add the weir 'tutorial' at branch 'Riv_RS1_1076' 1 km from the upstream point with a positive flow direction, crest level of 8.9 m AD, crest width of 8 m, and a correction coefficient of 0.9. (_Tip: check [tutorial HYDROLIB-core](https://deltares.github.io/HYDROLIB-core/tutorials/steps/)._)

In [13]:
# Answer to 13

# Create a weir
struc_weir = Weir(
    id="tutorial", 
    name="tutorial_weir", 
    branchid="Riv_RS1_1076", 
    chainage=1000.0, type="weir", 
    allowedflowdir=FlowDirection.positive, 
    crestlevel=8.9, 
    crestwidth=8.0, 
    corrcoef=0.9, 
    usevelocityheight=True
)

# Print crest level of this newly created weir
print(struc_weir.crestlevel)

# Add weir structure to other structures
fm.geometry.structurefile[0].structure.append(struc_weir)



8.9


Note that the code below would generate a new structure model and all the other structure will be lost. 
```python
fm.geometry.structurefile = [StructureModel(structure=[struc_weir])]
```
If you start from scratch, then the code above will be useful.

__EX14__: (a) Create a dimr model (b) from the MDU-model and (c+d) save the DIMR model.

In [18]:
# Answer to 14

# a) Create a dimr model
dimr = DIMR()

# b) Add the existing fm model from Ex.2 to this dimr model
dimr.component.append(
    FMComponent(name="MGB", workingDir=".", inputfile=fm.filepath, model=fm)
)

# c) Generate new directory
outputdimr = root/"output_dimr"
outputdimr.mkdir(parents=True, exist_ok=True)

# d) save dimr model
dimr.save(outputdimr)

WindowsPath('d:/dam_ar/unstruc_plannen/2021/20210000_HYDROLIB/N/E. Correspondence/Meetings/20211102_HYDROLIB-core_developer_training/getting_started/output_dimr/dimr_config.xml')